In [42]:
import os 
import re
import codecs
import _uniout 
import glob
import pymongo
from pymongo import MongoClient

def no_court(content):
    court_no = {u"臺北地方":100,u"士林地方":111,u"新北地方":220,u"宜蘭地方":260,u"基隆地方":200,u"桃園地方":320,u"新竹地方":300,u"苗栗地方":350,u"臺中地方":400,u"彰化地方":500,u"南投地方":540,u"臺南地方":700,u"高雄地方":800,u"花蓮地方":970,u"雲林地方":630,u"嘉義地方":602,u"臺東地方":950,u"屏東地方":900,u"澎湖地方":880,u"福建金門":890,u"福建連江":209,u"高雄少年":850}
    m = re.search(u'[  \d]*(灣|福建)(\S{4})法院(民事|家事|暫時|支付)(\S{2})',content)
    return str(court_no[m.group(2)])

def no_jurytype(content):
    jurytype_no = {u"婚":11,u"婚再":12,u"婚更":13,u"家聲":21,u"家訴":22,u"家抗":23,u"家陸許":24,u"司家聲":30,u"基家簡":25,u"重家訴":26,u"婚更一":27,u"婚更(一)":28,u"司家調":29,u"家移調":31,u"家簡":32,u"家調裁":33,u"家查":34,u"家聲抗":35,u"家調裁":36,u"家調補":37,u"司家補":38,u"家親聲":39,u"司家簡調":40,u"家簡上":41,u"家小":42,u"家救":43,u"家小上":44,u"家婚聲":45,u"家再":46,u"家他":47,u"補":90}
    m = re.search(u'(【裁判字號】)(.*?),(.*?),(\d{1,4})[^【】 ]',content)
    return m.group(2)+"_"+str(jurytype_no[m.group(3)])+"_"+m.group(4)

def judgedate(content):
    m = re.search(u'(【裁判日期】)(\d{6,7})',content)
    return m.group(2)

def courtname(content):
    m = re.search(u'[  \d]*(灣|福建)(\S{4})法院(民事|家事|暫時|支付)(\S{2})',content)
    return m.group(2)

def getPK(content):
    return no_court(content)+"_"+no_jurytype(content)+"_"+judgedate(content)
#make a connection to mongodb 
client = pymongo.MongoClient('localhost',27017)
#assign db_name to db
db = client.test
#make the collection in the db of test
collection = db['dday']
dday = db.dday
#read the judicial case in the directory

list1 = [u'壹',u'貳',u'參',u'肆']
list2 = [u'一',u'二',u'三',u'四']
list3 = [u'甲',u'乙',u'丙',u'丁']
for filename in glob.glob(u'C:/Users/BigData/Documents/Python/judgement/*.txt'):    
    with codecs.open( filename ,'r' ,'utf-8') as f :
        content = f.read() # 不能圖超過 2Mb        
        f.seek(0,0) #檔案指標移回檔案開頭
        lines = f.readlines()# 不能圖超過 2Mb
        cnt = 0
        lv = 0
        txt = u''
        #print lines[cnt]
        try:
            for i in lines:
                if lv != 0 or cnt > 70:
                    break
                m = re.search(u'([壹貳參肆一二三四甲乙丙丁]{1,2})、(原告.*?)',i)
                if m !=None:
                    n = m.group(1)
                    if n in list1:
                        lv = 1
                    if n in list2:
                        lv = 2
                    if n in list3:
                        lv = 3
                    for j in lines:
                        txt += lines[cnt]
                        cnt += 1
                        if lv == 1 and re.match(u'[壹貳參肆]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        if lv == 2 and re.match(u'[一二三四]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        if lv == 3 and re.match(u'[甲乙丙丁]{1,2}、.*?',lines[cnt]) !=None:
                            break
                else:
                    cnt += 1
            
        except:
            pass
        #print type(txt)
        #dic = {"_id":getPK(content),"juryno":no_jurytype(content),"judgedate":judgedate(content),"courtname":courtname(content),"plaintiff":txt}
        #insert to the collection of dday
        update = dday.update({"_id":getPK(content)},{"$set":{"plaintiff":txt}},upsert=False)
        for i in dday.find({"_id":getPK(content)}):
            print _uniout.unescape(str(i),'utf-8') 
        
    print '============================================================='

{u'judgedate': u'920530', u'juryno': u'92_11_129', u'_id': u'200_92_11_129_920530', u'courtname': u'基隆地方', u'plaintiff': u''}
{u'judgedate': u'1010221', u'juryno': u'100_11_10', u'_id': u'260_100_11_10_1010221', u'courtname': u'宜蘭地方', u'plaintiff': u'''二、原告起訴主張：被告為大陸地區人民，兩造於民國89年11月
    2日結婚，同年11月29日在台辦理結婚登記，婚後夫妻感情
    初尚融洽，然被告於93年11月1日出境後要再入境時，境管
    局因被告有賣淫案件，就不讓被告入境。兩造約從97年之後
    就沒有聯絡迄今，被告未履行同居，顯係惡意遺棄原告在繼
    續狀態中，爰依民法第1052條第1項第5款規定，請求判決離
    婚，並聲明：准原告與被告離婚。
'''}
{u'judgedate': u'1010118', u'juryno': u'100_11_101', u'_id': u'260_100_11_101_1010118', u'courtname': u'宜蘭地方', u'plaintiff': u'''壹、原告陳文才起訴主張：兩造於民國九十九年五月十九日在大
    陸地區結婚後，被告曾來臺與原告同住。詎被告於一百年六
    月十五日離家出走，經原告四處尋找均無所獲，現已失卻聯
    絡，是以被告未再返家與原告共同生活，亦未負擔家庭生活
    費用，顯見被告惡意遺棄原告於繼續狀態，爰依民法第一千
    零五十二條第一項第五款訴請判准兩造離婚而聲明如主文。
'''}
{u'judgedate': u'1010221', u'juryno': u'100_11_106', u'_id': u'260_100_11_106_1010221', u'courtname': u'宜蘭地方', u'plaintiff': u'''二、原告起訴主張：被告係大陸地區人民，兩造於民國91年11月
    25日在大陸地區陝西省結婚，並於91年12月16日在臺辦理結
    婚登記，嗣被告於92年

KeyError: u'\u53f0\u5317\u5730\u65b9'

In [3]:
import os 
import re
import codecs
import _uniout 
import glob
list1 = [u'壹',u'貳',u'參',u'肆']
list2 = [u'一',u'二',u'三',u'四']
list3 = [u'甲',u'乙',u'丙',u'丁']
for filename in glob.glob(u'./judicial/*.txt'):
    with codecs.open( filename ,'r' ,'utf-8') as f :
        f.seek(0,0) #檔案指標移回檔案開頭
        lines = f.readlines()# 不能圖超過 2Mb
        cnt = 0
        lv = 0
        print lines[cnt]
        try:
            for i in lines:
                m = re.search(u'([壹貳參肆一二三四甲乙丙丁]{1,2})、(原告.*?)',i)
                if m !=None:
                    n = m.group(1)
                    if n in list1:
                        lv = 1
                    if n in list2:
                        lv = 2
                    if n in list3:
                        lv = 3
                cnt += 1
                if lv == 1 and re.match(u'[壹貳參肆伍陸柒捌玖]{1,2}、.*?',lines[cnt]) !=None:
                    break                         
                if lv == 2 and re.match(u'[一二三四五六七八九]{1,2}、.*?',lines[cnt]) !=None:
                    break       
                if lv == 3 and re.match(u'[甲乙丙丁戊己庚辛壬]{1,2}、.*?',lines[cnt]) !=None:
                    break
            while True:
                if re.search(u'.*?應堪採信.*?',lines[cnt]) != None or re.search(u'.*?未成年子女之最佳利益.*?',lines[cnt]) != None or re.search(u'.*?不足採信.*?',lines[cnt]) != None:
                    while (cnt > 0):
                        if lv == 1 and re.match(u'[壹貳參肆伍陸柒捌玖]{1,2}、.*?',lines[cnt]) !=None:
                            break                         
                        if lv == 2 and re.match(u'[一二三四五六七八九]{1,2}、.*?',lines[cnt]) !=None:
                            break       
                        if lv == 3 and re.match(u'[甲乙丙丁戊己庚辛壬]{1,2}、.*?',lines[cnt]) !=None:
                            break
                        cnt -= 1
                    while (cnt < len(lines)):
                        print lines[cnt]
                        cnt += 1
                        if lv == 1 and re.match(u'[壹貳參肆伍陸柒捌玖]{1,2}、.*?',lines[cnt]) !=None:
                            break                         
                        if lv == 2 and re.match(u'[一二三四五六七八九]{1,2}、.*?',lines[cnt]) !=None:
                            break       
                        if lv == 3 and re.match(u'[甲乙丙丁戊己庚辛壬]{1,2}、.*?',lines[cnt]) !=None:
                            break                                    
                else:
                    cnt += 1
                                    
        except:
            pass
        
    print '============================================================='

【裁判字號】100,婚,100

【裁判字號】100,婚,107

五、原告主張兩造結婚後，因成長環境背景的差異，造成諸多觀

    念、想法差距甚大，於新婚赴海外度蜜月時，即因購買贈送

    親友的禮品、觀看風景等事而發生爭吵，被告並在20幾個旅

    行團團員的面前吼原告，且婚後生活兩造仍常因消費、交友

    及日常生活習慣等事宜意見相左，而大、小爭執不斷，最後

    甚至在居住的處所，各自劃出生活的居住範圍之事實，業據

    證人錢馨琪於本院證述：當時兩造交往半年就說要結婚，當

    時伊沒有同意，伊認為雙方背景生活環境差很大。原告是備

    受家庭疼愛，生活優渥，被告是南部的小孩，伊認為兩造價

    值觀差太多，但是被告表示29歲不可以結婚，伊有問被告是

    不是可以接受伊之前提出的疑問，被告說可以，所以才結婚

    。但是不到1 年就吵要離婚，伊基於勸合不勸離，一開始伊

    都是罵伊自己的女兒，後來被告常常打電話給伊，都是在說

    原告不好的地方，伊慢慢去思考在結婚的當時兩造居住的房

    子，是被告家裡支出的沒有錯，但是家用及車子都是原告負

    擔，兩造間有很多的問題。被告的爸爸也有打電話給伊，說

    兩造不如就離婚好了。有1 次兩造吵架，被告用電話砸原告

    ，原告躲到房間裡面，原告打電話向伊哭訴。伊是希望兩造

    可以好聚好散，不要撕破臉，被告一天到晚打給伊抱怨原告

    ，伊認為兩造已經沒有共同生活的必要了等語（見本院卷第

    96頁背面、97頁）。參以被告亦於本院陳述：兩造剛結婚的

    時候，對於金錢觀確實有所不同，2 人在蜜月期間有爭吵，

    但是12年前發生的事情，以後在別人的面前就都沒有了，只

    有私下就金錢觀的事情有所討論。家裡的費用都是伊在支付

    ，原告雖然有在工作，但是她自己賺錢自己花，原告的收入

    不算少，伊會覺得比較累。原告如果找伊出國去玩，就要花

    伊的錢，如果在臺灣，1 個月約2 、3 次出遊，1 次出去玩

    大約2 、3 千，出去就由伊付錢，沒有再給原告家用。伊1

    個月的月薪約6 萬，房貸2 、3 萬。我們兩個都不愛乾淨，

    也不愛打掃。伊從剛認識她的時候，她就這